In [34]:
from google.cloud import bigquery
from google.oauth2 import service_account
import json

In [2]:
credentials = service_account.Credentials.from_service_account_file(
'../../github-issue-data-extraction-key.json')

project_id = 'github-issue-data-extraction'
client = bigquery.Client(credentials= credentials, project=project_id)

In [88]:
# yyyymmdd
date = '20190101'

In [86]:
response = client.query(f"""SELECT JSON_EXTRACT(payload, '$.issue.title') as title,
                            JSON_EXTRACT(payload, '$.issue.body') as body,
                            JSON_EXTRACT(payload, '$.issue.html_url') as url,
                            JSON_EXTRACT(payload, '$.issue.user.login') as actor
                            FROM githubarchive.day.{date}
                            WHERE type = 'IssuesEvent' AND JSON_EXTRACT(payload, '$.action') = '"opened"' """)

In [96]:
df = response.to_dataframe()

In [97]:
# little bit of preprocessing

def remove_quotes(string):
    if type(string) == str:
        return string[1:-1]
    else:
        return string

for col in df.columns:
    df[col] = df[col].apply(remove_quotes)
    
def is_bot(actor):
    if actor[-5:] == '[bot]':
        return True
    else:
        return False
    
df = df[~df[col].apply(is_bot)]